# Client Project: The Lab @ DC

## Project Title: {here}

### Authors: Kihoon Sohn, Brian Collins, Harsha Goonawardana, Priya Kakkar
- Cohorts of the Data Science Immersive, General Assembly @ Washington DC campus

In this notebook, we have Exploratory Data Analysis on the City Service Requests / ShotsSpotters datasets. **This is notebook 2 of 3.**

### Import Libraries

In [272]:
# import basic libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### Read CSVs

In [273]:
csr_train   = pd.read_csv('./assets/csr/csr_train.csv', low_memory=False)
csr_test    = pd.read_csv('./assets/csr/csr_test.csv', low_memory=False)
shots_train = pd.read_csv('./assets/mpd/shots_train.csv', low_memory=False)
shots_test  = pd.read_csv('./assets/mpd/shots_test.csv', low_memory=False)

##### check null values and basic info on the datasets

In [274]:
csr_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231233 entries, 0 to 1231232
Data columns (total 30 columns):
X                             1231233 non-null float64
Y                             1231233 non-null float64
OBJECTID                      1231233 non-null int64
SERVICECODE                   1231233 non-null object
SERVICECODEDESCRIPTION        1231233 non-null object
SERVICETYPECODEDESCRIPTION    1230379 non-null object
ORGANIZATIONACRONYM           1231232 non-null object
SERVICECALLCOUNT              1231233 non-null int64
ADDDATE                       1231233 non-null object
RESOLUTIONDATE                1145187 non-null object
SERVICEDUEDATE                1218530 non-null object
SERVICEORDERDATE              1231233 non-null object
INSPECTIONFLAG                1231233 non-null object
INSPECTIONDATE                434130 non-null object
INSPECTORNAME                 40361 non-null object
SERVICEORDERSTATUS            1230380 non-null object
STATUS_CODE               

In [275]:
csr_train.isnull().sum().sort_values(ascending=False)

INSPECTORNAME                 1190872
INSPECTIONDATE                 797103
DETAILS                        444580
MARADDRESSREPOSITORYID         189162
STATUS_CODE                    151801
RESOLUTIONDATE                  86046
CITY                            50324
STATE                           50324
STREETADDRESS                   49730
SERVICEDUEDATE                  12703
WARD                             6221
PRIORITY                         2677
SERVICETYPECODEDESCRIPTION        854
SERVICEORDERSTATUS                853
ZIPCODE                            16
ORGANIZATIONACRONYM                 1
SERVICEREQUESTID                    0
XCOORD                              0
INSPECTIONFLAG                      0
SERVICEORDERDATE                    0
YCOORD                              0
LATITUDE                            0
ADDDATE                             0
SERVICECALLCOUNT                    0
LONGITUDE                           0
SERVICECODEDESCRIPTION              0
SERVICECODE 

In [276]:
shots_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28343 entries, 0 to 28342
Data columns (total 7 columns):
ID           28339 non-null object
Type         28343 non-null object
Date         28343 non-null object
Time         28343 non-null object
Source       28343 non-null object
Latitude     28343 non-null float64
Longitude    28343 non-null float64
dtypes: float64(2), object(5)
memory usage: 1.5+ MB


In [277]:
shots_train.isnull().sum().sort_values(ascending=False)

ID           4
Longitude    0
Latitude     0
Source       0
Time         0
Date         0
Type         0
dtype: int64

### Basic EDAs and Data Cleaning

##### Basic EDAs on Shots dataset

In [278]:
shots_train.head()

,ID,Type,Date,Time,Source,Latitude,Longitude
0,5D39700,Multiple_Gunshots,2014-01-01,00:00:02,WashingtonDC5D,38.917,-77.012
1,5D39701,Multiple_Gunshots,2014-01-01,00:00:06,WashingtonDC5D,38.917,-77.002
2,5D39702,Multiple_Gunshots,2014-01-01,00:00:07,WashingtonDC5D,38.917,-76.987
3,7D119445,Multiple_Gunshots,2014-01-01,00:00:10,WashingtonDC7D,38.823,-77.000
4,1D55993,Multiple_Gunshots,2014-01-01,00:00:10,WashingtonDC1D,38.893,-76.993


In [279]:
shots_train.shape

(28343, 7)

In [280]:
shots_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28343 entries, 0 to 28342
Data columns (total 7 columns):
ID           28339 non-null object
Type         28343 non-null object
Date         28343 non-null object
Time         28343 non-null object
Source       28343 non-null object
Latitude     28343 non-null float64
Longitude    28343 non-null float64
dtypes: float64(2), object(5)
memory usage: 1.5+ MB


In [281]:
shots_train.isnull().sum().sort_values(ascending=False)

ID           4
Longitude    0
Latitude     0
Source       0
Time         0
Date         0
Type         0
dtype: int64

In [282]:
shots_train[shots_train.ID.isnull()]

,ID,Type,Date,Time,Source,Latitude,Longitude
17514,NaN,Multiple_Gunshots,2015-12-27,19:41:22,WashingtonDC5D,38.931,-76.970
20378,NaN,Gunshot_or_Firecracker,2016-06-28,11:41:36,WashingtonDC6D,38.894,-76.924
24456,NaN,Single_Gunshot,2017-02-22,17:10:13,WashingtonDC7D,38.841,-76.976
24691,NaN,Single_Gunshot,2017-03-15,20:08:03,WashingtonDC7D,38.840,-76.988


In [283]:
def shot_spot_preprocess(df):
    shots_train.set_index(['ID'], inplace=True)
    shots_train.Source = shots_train.Source.apply(lambda DC: DC.replace('WashingtonDC', ''))
    shots_train.Date = pd.to_datetime(shots_train.Date, infer_datetime_format=True)
    return df

shots_train = shot_spot_preprocess(shots_train)

In [284]:
# shots_train.to_csv('./assets/mpd/shots_train_preprocessed.csv')

In [285]:
shots_train.Source.value_counts()

7D    10342
6D     8407
5D     3216
4D     2701
1D     1954
3D     1723
Name: Source, dtype: int64

In [286]:
shots_train.Type.value_counts()

Multiple_Gunshots         15858
Single_Gunshot            10034
Gunshot_or_Firecracker     2451
Name: Type, dtype: int64

##### Basic EDAs on City Service Requests Datasets

In [287]:
csr_train.iloc[:5, :6]

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION
0,-76.972735,38.897957,463232,S0011,Alley Cleaning,Street Cleaning
1,-76.991907,38.922865,463233,S0321,Recycling Collection - Missed,Recycling
2,-76.970891,38.874749,463234,S0031,Bulk Collection,Bulk Collection
3,-77.022678,38.942819,463235,S0311,Rat Abatement,DOH
4,-77.048840,38.898960,463236,S0276,Parking Meter Repair,TOA


##### findings above
- X and Y seems to be closely related to Lats/Longs - drop!
- Is service code description the same as service type, just a little more information?
- Object ID can be indexed
- Service code can be grouped by with code description

In [288]:
csr_train.iloc[:5, 6:12]

,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,SERVICEDUEDATE,SERVICEORDERDATE
0,DPW,1,2014-01-02T13:27:40.000Z,2014-01-15T07:43:42.000Z,2014-02-18T13:27:40.000Z,2014-01-02T13:27:40.000Z
1,DPW,1,2014-01-02T13:46:57.000Z,2014-01-06T12:39:39.000Z,2014-01-06T13:46:57.000Z,2014-01-02T13:46:57.000Z
2,DPW,1,2014-01-02T13:57:46.000Z,2014-01-14T14:29:16.000Z,2014-01-23T13:57:46.000Z,2014-01-02T13:57:46.000Z
3,DOH,1,2014-01-02T13:43:20.000Z,NaN,2014-02-24T13:43:20.000Z,2014-01-02T13:43:20.000Z
4,DDOT,1,2014-01-02T16:00:59.000Z,2014-01-07T16:33:48.000Z,2014-01-09T16:00:59.000Z,2014-01-02T16:00:59.000Z


##### findings above

- Organization to know who is on the task.
- Service call count to see how many times a call is needed.
- Everything just needs to be set to datetime.
- Organization acronym / service call count can be dropped.

In [289]:
csr_train.iloc[:5, 12:18]

,INSPECTIONFLAG,INSPECTIONDATE,INSPECTORNAME,SERVICEORDERSTATUS,STATUS_CODE,SERVICEREQUESTID
0,N,NaN,NaN,CLOSED,NaN,14-00000654
1,N,2014-01-06T12:39:00.000Z,"Bryant, Kevin",CLOSED,NaN,14-00000686
2,N,NaN,NaN,CLOSED,NaN,14-00000707
3,N,NaN,NaN,OPEN,NaN,14-00000677
4,N,NaN,NaN,CLOSED,NaN,14-00000877


##### findings above
- Service order data good.  Needs to be broken down into Datetime.
- Inspection Flag, what does that mean?
- Remove Inspector Name
- What does Status code contain? A lot of NaN could be bad
- Service Order Status, Important maybe?

In [290]:
csr_train.iloc[:5, 18:24]

,PRIORITY,STREETADDRESS,XCOORD,YCOORD,LATITUDE,LONGITUDE
0,STANDARD,2301 BENNING ROAD NE,402365.36,136678.02,38.897950,-76.972732
1,STANDARD,1004 RHODE ISLAND AVENUE NE,400701.99,139442.62,38.922857,-76.991905
2,STANDARD,2333 FAIRLAWN AVENUE SE,402526.12,134101.82,38.874742,-76.970889
3,STANDARD,720 VARNUM STREET NW,398034.18,141657.91,38.942811,-77.022676
4,STANDARD,700 - 799 BLOCK OF 22ND STREET NW,395763.56,136790.11,38.898952,-77.048838


##### findings above
- Service Request ID seems unimportant
- What is the XCOORD and YCOORD?
- Street Address can help us find our quandrants. Do we also want the address or is Latitude and Longitude
- Priority - how many unique values are in there?

In [291]:
csr_train.iloc[:5, 24:31]

,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS
0,WASHINGTON,DC,20002.0,48983.0,Ward 7,There is some dumping in the rear of this addr...
1,WASHINGTON,DC,20018.0,76304.0,Ward 5,Has not been collected the past 4 weeks.
2,WASHINGTON,DC,20020.0,286919.0,Ward 7,"1 television, 2 vacuums, 1 boom box,"
3,WASHINGTON,DC,20011.0,249794.0,Ward 4,requesting ratb abatement
4,WASHINGTON,DC,20052.0,NaN,2,Broken Parking Meter


##### findings above
- Remove City and State
- clean up the Ward to just numbers
- Fix Zipcode to be int.
- Longitude is the same as the X column
- Details can be vectorized.
- What is MARADDRESSREPOSITORYID?

##### Check with some categorical columns' values and find typos or misspelled. 

In [292]:
csr_train.PRIORITY.value_counts()

STANDARD     1179004
URGENT         24585
EMERGNCY       23635
EMERGENCY       1189
PRIORITY         141
PRIOR003           1
ESCALATED          1
Name: PRIORITY, dtype: int64

##### Preprocessing

In [293]:
csr_train.columns

Index(['X', 'Y', 'OBJECTID', 'SERVICECODE', 'SERVICECODEDESCRIPTION',
       'SERVICETYPECODEDESCRIPTION', 'ORGANIZATIONACRONYM', 'SERVICECALLCOUNT',
       'ADDDATE', 'RESOLUTIONDATE', 'SERVICEDUEDATE', 'SERVICEORDERDATE',
       'INSPECTIONFLAG', 'INSPECTIONDATE', 'INSPECTORNAME',
       'SERVICEORDERSTATUS', 'STATUS_CODE', 'SERVICEREQUESTID', 'PRIORITY',
       'STREETADDRESS', 'XCOORD', 'YCOORD', 'LATITUDE', 'LONGITUDE', 'CITY',
       'STATE', 'ZIPCODE', 'MARADDRESSREPOSITORYID', 'WARD', 'DETAILS'],
      dtype='object')

In [294]:
# create preprocess function

# after the basic EDA above, decided to drop columns below
drop_cols = ['X', 'Y', 'ORGANIZATIONACRONYM', 'SERVICECALLCOUNT',
             'SERVICEDUEDATE', 'SERVICEORDERDATE', 'INSPECTIONFLAG',
             'INSPECTIONDATE', 'INSPECTORNAME',
             'STREETADDRESS', 'XCOORD', 'YCOORD', 'CITY', 'STATE',
             'MARADDRESSREPOSITORYID', 'DETAILS']


def crimespot_preprocess(df):
    # Removing unused or redundent information
    csr_train.drop(drop_cols, axis=1, inplace=True)
    
    # Easier to work with lowercase columns
    csr_train.columns = map(str.lower, csr_train.columns) 

    # replace values
    csr_train.priority = csr_train.priority.replace("EMERGNCY", "EMERGENCY")
        
    # treat zipcode as string and strip
    csr_train.zipcode = csr_train.zipcode.astype(str).str.strip().str.strip('.0')

    # (KS) make single line to combine Brian's on 'Ward'
    csr_train.ward = csr_train.ward.astype(str).map(lambda x: x.strip('Ward')).str.strip().str.strip('.0')

    # create binary classfication column by resolution date info
    csr_train['resolved'] = [0 if x == True else 1 for x in csr_train['resolutiondate'].isnull()]

    # fill nan values to 0
    csr_train.resolutiondate.fillna('0', inplace=True)
    
    # clean up datetime related data
    timestamp = ['adddate', 'resolutiondate']
    for x in timestamp:
        csr_train[x] = csr_train[x].astype(str).map(lambda x: x.strip('Z').replace('T', ' ')).astype('datetime64[ns]')
        
    # calculate the time difference between resolutiondate - adddate by hours
    csr_train['turnover'] = (csr_train['resolutiondate']-csr_train['adddate']).astype('timedelta64[h]')*csr_train['resolved']

    csr_train['servicecodedescription'] = [x.lower() for x in csr_train['servicecodedescription']]

    return df

csr_train = crimespot_preprocess(csr_train)

In [295]:
# no resolution date gets 0

csr_train['resolved'].value_counts()

1    1145187
0      86046
Name: resolved, dtype: int64

In [296]:
# turnover stats

csr_train['turnover'].describe().apply(lambda x: format(x, 'f'))

count    1231233.000000
mean         571.527879
std         1747.934413
min            0.000000
25%            6.000000
50%           72.000000
75%          268.000000
max        28247.000000
Name: turnover, dtype: object

In [297]:
csr_train.dtypes

objectid                               int64
servicecode                           object
servicecodedescription                object
servicetypecodedescription            object
adddate                       datetime64[ns]
resolutiondate                datetime64[ns]
serviceorderstatus                    object
status_code                           object
servicerequestid                      object
priority                              object
latitude                             float64
longitude                            float64
zipcode                               object
ward                                  object
resolved                               int64
turnover                             float64
dtype: object

##### Spatial data (ward, zipcode)
- Initially, team planned to use `ward`, `zipcode` for anlaysis, however we decided to use geopandas to assign PSA to each values. Therefore, no needs to keep `ward`, `zipcode` here.

In [298]:
csr_train.ward.value_counts()

2      254937
6      214903
4      158945
5      153126
1      127995
7      119438
3      114738
8       80930
nan      6221
Name: ward, dtype: int64

- zipcode has bunch of spoiled values.

In [299]:
csr_train.zipcode.value_counts()

20002    144190
20011    123232
20001    119199
20019     90170
20009     80421
2002      66351
20007     61568
20003     61316
2001      51723
20016     48526
20032     37641
20018     37447
20008     34014
20005     33947
20017     33281
20015     31462
20037     30120
20036     29633
20024     29490
20012     28990
20006     22403
20004     19572
20052      4942
20415      1011
20059       966
2025        854
2024        636
20405       601
20201       553
20057       523
          ...  
-2207         1
18            1
20903         1
23294         1
60435         1
20705         1
20906         1
20886         1
-1254         1
28272         1
2051          1
-2103         1
2             1
22046         1
20746         1
16            1
22406         1
-4115         1
2122          1
9002          1
-1865         1
10533         1
36            1
20135         1
11            1
20613         1
22153         1
24577         1
-2326         1
31088         1
Name: zipcode, Length: 1

### Geopandas to PSA

In [300]:
import geopandas as gpd
from shapely.geometry import Point

In [301]:
geometry = [Point(xy) for xy in zip(csr_train['longitude'], csr_train['latitude'])]
csr_train.drop(['longitude', 'latitude'], axis=1, inplace=True)
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(csr_train, geometry=geometry, crs=crs)

In [302]:
psa = gpd.read_file('./assets/Police_Service_Areas.geojson')

In [303]:
psa = psa[['PSA','geometry']]

In [304]:
csr_train = gpd.sjoin(gdf, psa, how='left', op='within')

In [305]:
csr_train.head()

,objectid,servicecode,servicecodedescription,servicetypecodedescription,adddate,resolutiondate,serviceorderstatus,status_code,servicerequestid,priority,zipcode,ward,resolved,turnover,geometry,index_right,PSA
0,463232,S0011,alley cleaning,Street Cleaning,2014-01-02 13:27:40,2014-01-15 07:43:42.000000000,CLOSED,NaN,14-00000654,STANDARD,20002,7,1,306.0,POINT (-76.97273246 38.89794972),22.0,507.0
1,463233,S0321,recycling collection - missed,Recycling,2014-01-02 13:46:57,2014-01-06 12:39:39.000000000,CLOSED,NaN,14-00000686,STANDARD,20018,5,1,94.0,POINT (-76.99190473 38.92285708),42.0,504.0
2,463234,S0031,bulk collection,Bulk Collection,2014-01-02 13:57:46,2014-01-14 14:29:16.000000000,CLOSED,NaN,14-00000707,STANDARD,2002,7,1,288.0,POINT (-76.97088871 38.87474188),10.0,605.0
3,463235,S0311,rat abatement,DOH,2014-01-02 13:43:20,1753-08-29 22:43:41.128654848,OPEN,NaN,14-00000677,STANDARD,20011,4,0,-0.0,POINT (-77.02267596 38.94281117),46.0,407.0
4,463236,S0276,parking meter repair,TOA,2014-01-02 16:00:59,2014-01-07 16:33:48.000000000,CLOSED,NaN,14-00000877,STANDARD,20052,2,1,120.0,POINT (-77.04883778217619 38.8989521053866),23.0,207.0


In [306]:
csr_train.reset_index()

,index,objectid,servicecode,servicecodedescription,servicetypecodedescription,adddate,resolutiondate,serviceorderstatus,status_code,servicerequestid,priority,zipcode,ward,resolved,turnover,geometry,index_right,PSA
0,0,463232,S0011,alley cleaning,Street Cleaning,2014-01-02 13:27:40,2014-01-15 07:43:42.000000000,CLOSED,NaN,14-00000654,STANDARD,20002,7,1,306.0,POINT (-76.97273246 38.89794972),22.0,507.0
1,1,463233,S0321,recycling collection - missed,Recycling,2014-01-02 13:46:57,2014-01-06 12:39:39.000000000,CLOSED,NaN,14-00000686,STANDARD,20018,5,1,94.0,POINT (-76.99190473 38.92285708),42.0,504.0
2,2,463234,S0031,bulk collection,Bulk Collection,2014-01-02 13:57:46,2014-01-14 14:29:16.000000000,CLOSED,NaN,14-00000707,STANDARD,2002,7,1,288.0,POINT (-76.97088871 38.87474188),10.0,605.0
3,3,463235,S0311,rat abatement,DOH,2014-01-02 13:43:20,1753-08-29 22:43:41.128654848,OPEN,NaN,14-00000677,STANDARD,20011,4,0,-0.0,POINT (-77.02267596 38.94281117),46.0,407.0
4,4,463236,S0276,parking meter repair,TOA,2014-01-02 16:00:59,2014-01-07 16:33:48.000000000,CLOSED,NaN,14-00000877,STANDARD,20052,2,1,120.0,POINT (-77.04883778217619 38.8989521053866),23.0,207.0
5,5,463237,S0031,bulk collection,Bulk Collection,2014-01-02 16:25:59,2014-01-15 13:35:29.000000000,CLOSED,NaN,14-00000906,STANDARD,20011,4,1,309.0,POINT (-76.99737579000001 38.95757042),53.0,406.0
6,6,463238,S0276,parking meter repair,TOA,2014-01-02 16:14:47,2014-01-06 10:33:16.000000000,CLOSED,NaN,14-00000895,STANDARD,20001,2,1,90.0,POINT (-77.0239673421657 38.9003384044116),19.0,102.0
7,7,463239,S05SL,streetlight repair investigation,TOA,2014-01-02 16:49:52,2014-01-07 03:48:06.000000000,OVERDUE CLOSED,NaN,14-00000929,URGENT,20016,3,1,106.0,POINT (-77.08433844 38.95491079),51.0,202.0
8,8,463240,S0031,bulk collection,Bulk Collection,2014-01-02 16:48:08,2014-01-14 11:50:46.000000000,CLOSED,NaN,14-00000927,STANDARD,2002,7,1,283.0,POINT (-76.96597465000001 38.86834567),9.0,607.0
9,9,463241,S0441,trash collection - missed,Trash Collection,2014-01-02 16:06:49,2014-01-06 12:18:00.000000000,CLOSED,NaN,14-00000885,STANDARD,20011,4,1,92.0,POINT (-77.01967236999999 38.94713416),46.0,407.0


In [307]:
csr_train.PSA.isnull().sum()

11

In [308]:
csr_train.columns

Index(['objectid', 'servicecode', 'servicecodedescription',
       'servicetypecodedescription', 'adddate', 'resolutiondate',
       'serviceorderstatus', 'status_code', 'servicerequestid', 'priority',
       'zipcode', 'ward', 'resolved', 'turnover', 'geometry', 'index_right',
       'PSA'],
      dtype='object')

In [309]:
csr_train.drop(['index_right', 'geometry', 'ward', 'zipcode'], axis=1, inplace=True)

In [310]:
csr_train.set_index('objectid', inplace=True)

### Clean servicecode / description

In [311]:
csr_train.columns

Index(['servicecode', 'servicecodedescription', 'servicetypecodedescription',
       'adddate', 'resolutiondate', 'serviceorderstatus', 'status_code',
       'servicerequestid', 'priority', 'resolved', 'turnover', 'PSA'],
      dtype='object')

In [312]:
# 'service code' and 'service code description' mismatches its values

print(csr_train['servicecode'].nunique())
print(csr_train['servicecodedescription'].nunique())

164
214


In [313]:
# obviously, 'service code description' needs to be cleaned and merged by one adequate description.
# (e.g. dead animal collection v. dead animal pickup) 

csr_train.groupby(["servicecode", "servicecodedescription"]).count().head()

servicetypecodedescription  \
servicecode servicecodedescription                                        
11          dead animal collection                                 6927   
            dead animal pickup                                     3506   
BEDBUGS     bed bugs                                                 21   
BICYCLE     abandoned bicycle                                      2556   
C62313      christmas tree removal-seasonal                         549   

                                             adddate  resolutiondate  \
servicecode servicecodedescription                                     
11          dead animal collection              6930            6930   
            dead animal pickup                  3506            3506   
BEDBUGS     bed bugs                              21              21   
BICYCLE     abandoned bicycle                   2556            2556   
C62313      christmas tree removal-seasonal      549             549   

                                             serviceorderstatus  status_code  \
servicecode servicecodedescription                                             
11          dead animal collection                         6927         6930   
            dead animal pickup                             3506         2704   
BEDBUGS     bed bugs                                         21           21   
BICYCLE     abandoned bicycle                              2556         2556   
C62313      christmas tree removal-seasonal                 549          549   

                                             servicerequestid  priority  \
servicecode servicecodedescription                                        
11          dead animal collection                       6930      6927   
            dead animal pickup                           3506      3506   
BEDBUGS     bed bugs                                       21        21   
BICYCLE     abandoned bicycle                            2556      1532   
C62313      christmas tree removal-seasonal               549       549   

                                             resolved  turnover   PSA  
servicecode servicecodedescription                                     
11          dead animal collection               6930      6930  6930  
            dead animal pickup                   3506      3506  3506  
BEDBUGS     bed bugs                               21        21    21  
BICYCLE     abandoned bicycle                    2556      2556  2556  
C62313      christmas tree removal-seasonal       549       549   549

In [314]:
# courtesy of Ben Shaver
# replace duplicated code descriptions in each service code.

foo = csr_train.groupby(["servicecode", "servicecodedescription"]).count()
bar = foo.reset_index().groupby('servicecode')['servicecodedescription'].count().sort_values(ascending=False)

print(bar[bar > 1].index)

for code in bar[bar > 1].index:
    replacement = csr_train[csr_train['servicecode'] == code]['servicecodedescription'].value_counts().argmax()
    csr_train.loc[csr_train['servicecode'] == code, 'servicecodedescription'] = replacement

Index(['S05AL', 'DDOCONRU', 'OUTODOR', 'SIGTRAMA', 'SPSTDAMA', 'DDENIDTI',
       'S0476', 'DDOTCITA', 'FOAMBAN', 'POWEOUTA', 'DMINSTIS', 'S0406',
       'RECCONRE', '11', 'S0021', 'S0322', 'EMERPOWE', 'S0311', 'DMVOFFSE',
       'S0081', 'DOEEGEC', 'S0166', 'S0217', 'S0216', 'S0196', 'F3242',
       'S0466', 'SAENCUFO', 'SRC01', 'HOMYDRCO', 'CONFSIGN', 'HMDCP',
       'CSRREPIS', 'DC31SERE', 'KIT', 'STRECUFO', 'DDBALATI', 'MARKINST',
       'S0185', 'SO392', 'MARKMODI', 'SCCRGUPR', 'MARKMAIN', 'MARKREMO',
       'SIGNMISS', 'SIDEWLKEXP'],
      dtype='object', name='servicecode')


/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.


In [318]:
csr_train['servicecodedescription'].value_counts().head(20)

parking meter repair                      223577
bulk collection                           183118
parking enforcement                       126732
pothole                                    46280
streetlight repair investigation           43696
emergency no-parking verification          42541
trash collection - missed                  34257
alley cleaning                             30823
sanitation enforcement                     29834
container removal                          29606
residential parking permit violation       25529
recycling collection - missed              23217
street cleaning                            20908
illegal dumping                            20510
roadway signs                              18385
sidewalk repair                            18188
tree inspection                            17287
abandoned vehicle - on public property     17063
graffiti removal                           15747
tree pruning                               14256
Name: servicecodedes

In [342]:
# map descriptions into small numbers of category

mapping = {
    'parking meter repair': 'parking meter repair',
    'bulk collection': 'collection',
    'parking enforcement': 'parking enforcement',
    'pothole': 'maintenance', 
    'streetlight repair investigation': 'light repair',
    'emergency no-parking verification': 'parking enforcement',
    'trash collection - missed': 'collection',
    'alley cleaning': 'street cleaning',
    'sanitation enforcement' : 'sanitation',
    'container removal': 'collection',
    'residential parking permit violation': 'parking enforcement', 
    'recycling collection - missed': 'collection',
    'street cleaning': 'street cleaning', 
    'illegal dumping': 'dumping', 
    'roadway signs': 'signs',
    'sidewalk repair': 'maintenance', 
    'tree inspection': 'tree related',
    'abandoned vehicle - on public property': 'maintenance', 
    'graffiti removal': 'graffiti removal',
    'tree pruning': 'tree related', 
    'rodent inspection and treatment': 'pesticide',
    'snow/ice removal': 'snow related',       
    'tree planting': 'tree related', 
    'tru report': 'report', 
    'out of state parking violation (rosa)': 'parking enforcement',
    'dead animal collection': 'collection', 
    'traffic signal issue': 'signs',
    'dmv - drivers license/id issues': 'dmv related',
    'sidewalk shoveling enforcement exemption': 'maintenance', 
    'street repair': 'maintenance',
    'tree removal': 'tree related', 
    'alleylight repair investigation': 'light repair',
    'yard waste - missed': 'collection', 
    'sign replacement': 'signs',
    'residential snow removal (servedc)': 'snow related',
    'dmv - vehicle registration issues': 'dmv related', 
    'recycling cart delivery': 'collection',
    'trash cart - delivery': 'collection', 
    'alley repair': 'maintenance', 
    'supercan - delivery': 'collection',
    'abandoned vehicle - on private property': 'safety', 
    'grass and weeds mowing': 'maintenance',
    'sign new investigation': 'signs', 
    'fems - community events': 'maintenance', 
    'vacant lot': 'transportation',
    'traffic safety investigation': 'safety', 
    'utility repair investigation': 'maintenance',
    'abandoned bicycle': 'bicycle', 
    'bicycle issues': 'bicycle', 
    'curb and gutter repair': 'maintenance',
    'dc government information': 'dc gov',
    'leaf season collection': 'collection',
    'fems - smoke alarm application': 'maintenance', 
    'how is my driving - complaint': 'transportation',
    'public space litter can-collection': 'collection', 
    'roadway striping / markings': 'maintenance',
    'public space litter can- installation/removal/repair': 'maintenance',
    'dmv - vehicle title issues': 'dmv related',
    'dmv - copy of ticket': 'dmv related',
    'supercan - repair': 'maintenance', 
    'sign removal investigation': 'signs',
    'snow removal complaints for sidewalks': 'snow related', 
    'trash cart repair': 'maintenance',
    'illegal poster': 'maintenance', 
    'marking maintenance': 'maintenance',
    'doee - general environmental concerns': 'safety', 
    'street sweeping': 'maintenance',
    'dmv - forms, applications, and manuals request': 'dmv related',
    'doee - construction – erosion runoff': 'maintenance', 
    'bus/rail issues': 'transportation',
    'dmv - online processing issues': 'dmv related', 
    'resident parking permit': 'parking enforcement',
    'recycling cart - repair': 'maintenance', 
    'signs conflicting': 'signs',
    'dpw correspondence tracking': 'dc gov',
    'christmas tree removal-seasonal': 'tree related',
    'child safety seat program': 'safety', 
    'dmv - refunds - tickets': 'dmv related',
    '311force reported issues': 'dc gov', 
    'insect treatment': 'pesticide',
    'christmas tree removal - seasonal': 'tree related', 
    'insects': 'pesticide', 
    'dmv - hearings': 'dmv related',
    'dmv - vehicle insurance lapse': 'dmv related', 
    'dmv - processing center manager': 'dmv related',
    'wire down/power outage': 'maintenance', 
    'doee - nuisance odor complaints': 'maintenance',
    'emergency - trees': 'tree related', 
    'marking modification': 'maintenance',
    'dmv - ticket payment dispute': 'dmv related',
    'school crossing guard': 'safety',
    'safe routes to school': 'safety', 
    'doee - engine idling tips': 'maintenance',
    'dmv - driver and vehicle services refund': 'dmv related',
    'dmv - adjudication supervisor': 'dmv related', 
    'hypothermia shelter information': 'safety',
    'snow metro bus shelter/stop': 'snow related',
    'graffiti removal - paint voucher request': 'graffiti removal',
    'dmv - driver record issues': 'dmv related', 
    'dmv - drivers license/id reinstatement': 'dmv related',
    'fems - fire safety education': 'safety', 
    'dcra - grass and weeds': 'safety',
    'dmv - etims ticket alert services issues': 'dmv related', 
    'snow towing': 'snow related',
    'homeless services - winter/hypothermia season': 'safety',
    'how is my driving - compliment': 'transportation', 
    'recycling- information request': 'maintenance',
    'ouc nye test': 'dc gov', 
    'dmv - vehicle inspection issues': 'dmv related',
    'emergency - power outage/wires down': 'maintenance', 
    'parks and recreation': 'maintenance',
    'ddot citation': 'dc gov', 
    'light-light pole': 'light repair', 
    'illegal fireworks': 'safety',
    'dmv - appeal': 'dmv related', 
    'dmv - offset tracking': 'dmv related', 
    'marking removal': 'maintenance',
    'dcra - trash and debris': 'maintenance', 
    'doee - ban on foam food containers': 'safety',
    'ddoe - bag law tips': 'maintenance', 
    'homeless encampment': 'safety',
    'snow ticket reimbursement': 'snow related', 
    'yard waste - missed - customer follow-up': 'collection',
    'light-infrastructure': 'light repair',
    'recycling collection - missed - customer follow-up': 'collection',
    'light-tunnel/underpass light repair': 'light repair', 
    'recycling - commercial only': 'maintenance',
    'trash collection - missed customer follow-up': 'collection',
    'dc 311 service requests': 'dc gov', 
    'dcra - vacant building': 'maintenance', 
    'snow other': 'snow related',
    'bed bugs': 'pesticide', 
    'fems - honor guard': 'dc gov', 
    'dmv - ticket ombudsman': 'dmv related', 
    'streetcar': 'maintenance',
    'ticket ombudsman': 'maintenance', 
    'light-overhead guide sign lighting repair': 'light repair',
    'dcra - misc': 'maintenance', 
    'sanitation enforcement - customer follow-up': 'maintenance',
    'hoarding': 'maintenance',
    'homeless services - hypothermia/cold/winter - protection items': 'safety',
    'trash container - delivery - customer follow-up': 'maintenance',
    'graffiti removal - customer follow-up': 'graffiti removal', 
    'fems - 20/20 vision plan': 'safety',
    'recycling - school program': 'safety', 
    'emergency - flooding': 'safety',
    'recycling container delivery - customer follow-up': 'collection',
    'dds - serious medication error': 'safety', 
    'emergency - senior assistance': 'safety',
    'supercan - repair - customer follow-up': 'maintenance',
    'bulk collection - unscheduled': 'collection',
    'homeless services - hypothermia/cold/winter - safety checks': 'safety',
    'dhs - iris update': 'maintenance', 
    'dcra - zoning': 'maintenance',
    'report invalid address to gis dept': 'safety',
    'dds - theft of personal property': 'safety',
    'emergency - supplies': 'maintenance',
    'school transit subsidy program': 'safety',
    'homeless services - hypothermia/cold/winter - transport to shelter': 'safety',
    'emergency - heating and cooling': 'maintenance',
    'signs - conflicting': 'signs',
    'survey sr type': 'dc gov'
}

In [348]:
csr_train['servicecodedescription'] = csr_train['servicecodedescription'].map(mapping)
csr_train['servicecodedescription'].value_counts()


collection              308906
parking meter repair    223577
parking enforcement     206177
maintenance             130821
tree related             54693
light repair             52047
street cleaning          51731
signs                    41402
sanitation               29834
dmv related              23652
snow related             21505
dumping                  20510
graffiti removal         15924
pesticide                14497
report                   11253
safety                   10129
transportation            5892
bicycle                   5052
dc gov                    3630
Name: servicecodedescription, dtype: int64

In [349]:
print(csr_train['servicecodedescription'].nunique())

19


In [350]:
csr_train.head()

,servicecode,servicecodedescription,servicetypecodedescription,adddate,resolutiondate,serviceorderstatus,status_code,servicerequestid,priority,resolved,turnover,PSA
objectid,,,,,,,,,,,,
463232,S0011,street cleaning,Street Cleaning,2014-01-02 13:27:40,2014-01-15 07:43:42.000000000,CLOSED,NaN,14-00000654,STANDARD,1,306.0,507.0
463233,S0321,collection,Recycling,2014-01-02 13:46:57,2014-01-06 12:39:39.000000000,CLOSED,NaN,14-00000686,STANDARD,1,94.0,504.0
463234,S0031,collection,Bulk Collection,2014-01-02 13:57:46,2014-01-14 14:29:16.000000000,CLOSED,NaN,14-00000707,STANDARD,1,288.0,605.0
463235,S0311,pesticide,DOH,2014-01-02 13:43:20,1753-08-29 22:43:41.128654848,OPEN,NaN,14-00000677,STANDARD,0,-0.0,407.0
463236,S0276,parking meter repair,TOA,2014-01-02 16:00:59,2014-01-07 16:33:48.000000000,CLOSED,NaN,14-00000877,STANDARD,1,120.0,207.0


In [351]:
csr_train['serviceorderstatus'].value_counts()

CLOSED                             1119897
IN-PROGRESS                          34178
RESOLUTN                             25125
OVERDUE CLOSED                       15505
LOCKED                                8207
OPEN                                  7400
DUPLICATE (OPENED)                    5621
DUPLICATE (CLOSED)                    5211
VOIDED                                3479
OVERDUE OPEN                          2598
CLOSED - INCOMPLETE INFORMATION       1628
NONWORK                               1303
IN PROGRESS                            167
CLOSE                                   46
DUPCLOSD                                 9
DISPUTE                                  6
Name: serviceorderstatus, dtype: int64